In [12]:
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import matplotlib
%matplotlib inline

In [2]:
# Read the driving log data
driving_log = pd.read_csv('data/driving_log.csv')

In [8]:
# loop through the Pandas dataframe and read the images whose paths
# are the rows of the dataframe.
images = []
measurements = []
correction = 0.2
for i in range(driving_log.shape[0]):
    center_image = cv2.imread(('data/'+driving_log['center'].iloc[i]).replace(' ', ''))
    left_image = cv2.imread(('data/'+driving_log['left'].iloc[i]).replace(' ', ''))
    right_image = cv2.imread(('data/'+driving_log['right'].iloc[i]).replace(' ', ''))
    center_steering_angle = driving_log['steering'].iloc[i]
    left_steering_angle = center_steering_angle + correction
    right_steering_angle = center_steering_angle - correction
    images.extend([center_image, np.fliplr(center_image.copy()), 
                  left_image,
                  right_image])
    measurements.extend([center_steering_angle, -center_steering_angle,
                        left_steering_angle,
                        right_steering_angle])

In [9]:
# Convert the lists to numpy arrays
X_train = np.array(images)
y_train = np.array(measurements)

In [10]:
# Print the number of images and their dimenstions & number of channels
print(X_train.shape)

(32144, 160, 320, 3)

In [13]:
# Construct the model
model = Sequential()
model.add(Lambda(lambda x: (x/255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
# Compile, fit and save the model
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10)
model.save('model.h5')

Train on 25715 samples, validate on 6429 samples
Epoch 1/10
25715/25715 [==============================] - 69s - loss: 0.0309 - val_loss: 0.0193
Epoch 2/10
25715/25715 [==============================] - 67s - loss: 0.0153 - val_loss: 0.0182
Epoch 3/10
25715/25715 [==============================] - 52s - loss: 0.0143 - val_loss: 0.0199
Epoch 4/10
25715/25715 [==============================] - 52s - loss: 0.0139 - val_loss: 0.0186
Epoch 5/10
25715/25715 [==============================] - 52s - loss: 0.0132 - val_loss: 0.0214
Epoch 6/10
25715/25715 [==============================] - 52s - loss: 0.0130 - val_loss: 0.0178
Epoch 7/10
25715/25715 [==============================] - 52s - loss: 0.0127 - val_loss: 0.0178
Epoch 8/10
25715/25715 [==============================] - 57s - loss: 0.0124 - val_loss: 0.0185
Epoch 9/10
25715/25715 [==============================] - 71s - loss: 0.0122 - val_loss: 0.0183
Epoch 10/10
25715/25715 [==============================] - 72s - loss: 0.0120 - val_los